<a href="https://colab.research.google.com/github/Saiviraj/Sai-viraj/blob/main/BCE_loss_and_Negative_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install torch

In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import torch as th
from torch.autograd import Variable as V
from torch import nn,optim
import numpy as np
import random

pd.set_option('display.max_columns', None)

%load_ext google.colab.data_table
from google.colab import data_table
from google.colab import drive

drive.mount("/content/ydrive/")
# os.chdir("Your Location here")

import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

data_folder = '/content/drive/'

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
Drive already mounted at /content/ydrive/; to attempt to forcibly remount, call drive.mount("/content/ydrive/", force_remount=True).
cuda


In [16]:
df = pd.read_csv(data_folder + 'Darknet.CSV', on_bad_lines='skip')
#current_directory = os.getcwd()
#print(current_directory)
print(df.shape)

df.iloc[:5]

FileNotFoundError: ignored

In [ ]:
print('Total Number of flows:',len(df['Flow ID'].unique()))
print('Obs per flow, 10 largets:\n', df['Flow ID'].value_counts().value_counts()[:10])

df['src-dst'] = df['Src IP']+'-'+df['Dst IP']
print('Number of unique flows per Flow ID:\n', df.groupby(['Flow ID'])['src-dst'].nunique().value_counts())

df['src-dst-dst port-protocol'] = df['Src IP']+'-'+df['Dst IP']+'-'+df['Dst Port'].astype(str)+'-'+df['Protocol'].astype(str)
print('Number of unique flow charactaristics per Flow ID:\n', df.groupby(['Flow ID'])['src-dst-dst port-protocol'].nunique().value_counts())

filtered_df = df[['Src IP', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp']].drop_duplicates()
filtered_df['Dst Port'] = filtered_df['Dst Port'].astype(str)
filtered_df['Protocol'] = filtered_df['Protocol'].astype(str)
filtered_df.iloc[:5]


In [ ]:
def gen_sameples_and_vocab_from_a_flow(df, flatten=True, source_ip = 'Src IP', dest_ip = 'Dst IP', proto = 'Protocol', dest_port = 'Dst Port'):
    """
    Create pairs from a Flow dataset and vocabulary
    """
    df = df.reset_index(drop=True)
    pairs = [(
        (df[source_ip][j],df[dest_ip][j]),
        (df[source_ip][j],df[dest_port][j]),
        (df[source_ip][j],df[proto][j]),
        (df[dest_port][j], df[dest_ip][j]),
        (df[proto][j], df[dest_ip][j])
     ) for j in range(len(df))]

    if flatten:
        pairs = [item for sublist in pairs for item in sublist]

    vocab = set(df[source_ip].tolist() + df[dest_ip].tolist() + df[dest_port].tolist() + df[proto].tolist())
    vocab_size = len(vocab)
    word_to_ind = {word:idx for idx,word in enumerate(vocab)}
    ind_to_word = {idx:word for idx,word in enumerate(vocab)}

    return pairs, vocab, vocab_size, word_to_ind, ind_to_word

pairs, vocab, vocab_size, word_to_ind, ind_to_word = gen_sameples_and_vocab_from_a_flow(filtered_df)

print(f'Vocabulary size: {vocab_size}')
print('First five elements:\n', list(vocab)[:5])
print('First five nested pairs:')
pairs[:10]



In [ ]:
corpus = pairs
voc_size = vocab_size

print(voc_size)
corpus[:5]

In [ ]:
t = 60
power_parameter = 0.75
ns_table_size = 100000

def make_ns_table_ips(pairs, t=60, power_parameter=0.75, ns_table_size=100000):
  """
  pairs - pairs of entities. Probably (ip_source,ip_destination) in our real data.
  t - threshold variable from the paper.
  """
  from collections import Counter
  import operator

  flat_list_of_pairs = [item for sublist in pairs for item in sublist]
  counter = Counter(flat_list_of_pairs)

  # replace all the counts for cases where #.times of appearence < t --> 0
  increasingly_sorted_counter = dict(sorted(counter.items(), key=operator.itemgetter(1)))
  increasingly_sorted_counter_keys = list(increasingly_sorted_counter.keys())
  increasingly_sorted_counter_values = np.array(list(increasingly_sorted_counter.values()))
  increasingly_sorted_counter_values[increasingly_sorted_counter_values<t] = 0
  increasingly_sorted_counter_thresholded = {i : j for i, j in zip(increasingly_sorted_counter_keys, increasingly_sorted_counter_values)}
  decreasingly_sorted_counter_thresholded = dict(sorted(increasingly_sorted_counter_thresholded.items(), key=operator.itemgetter(1),reverse=True))

  # convert to frequencies rather than counts
  # S = np.sum(np.array(list(decreasingly_sorted_counter_thresholded.values())))
  S = 1
  decreasingly_sorted_counter_thresholded_freq_values = [decreasingly_sorted_counter_thresholded[k]/S for k in decreasingly_sorted_counter_thresholded.keys()]
  decreasingly_sorted_counter_thresholded_keys = list(decreasingly_sorted_counter_thresholded.keys())

  # Create negative_sampling table
  # The negative sampling probabilities are proportional to the frequencies
  # to the power of a constant (typically 0.75).
  freqs_sorted = [(decreasingly_sorted_counter_thresholded_keys[j], decreasingly_sorted_counter_thresholded_freq_values[j]) for j in range(len(decreasingly_sorted_counter_thresholded_keys))]
  ns_table = {}
  sum_freq = 0
  for w, freq in freqs_sorted:
      ns_freq = freq ** power_parameter
      ns_table[w] = ns_freq
      sum_freq += ns_freq

  # Convert the negative sampling probabilities to integers, in order to make
  # sampling a bit faster and easier.
  # We return a list of tuples consisting of:
  # - the word
  # - its frequency in the training data
  # - the number of positions reserved for this word in the negative sampling table
  scaler = ns_table_size / sum_freq
  results = [(w, freq, int(round(ns_table[w]*scaler))) for w, freq in freqs_sorted]
  return results



In [ ]:
ns_table = make_ns_table_ips(pairs)

print('Sum:', np.sum([x[2] for x in ns_table]))
ns_table[:5]

In [ ]:
k = 6

unflattened_list = [[x[0]]*x[2] for x in ns_table]
flattened_list = np.array([item for sublist in unflattened_list for item in sublist])
false_entities = np.random.choice(flattened_list, size=k)


In [ ]:
pairs_batch = pairs[:16]
pairs_batch

In [ ]:
class ContextGenerator:
  def __init__(self, ns_table, k=20):

    # Cerate a repeated list of negative samples, using ns_table
    unflattened_list = [[x[0]]*x[2] for x in ns_table]
    flattened_list = np.array([item for sublist in unflattened_list for item in sublist])
    self.ns_table = flattened_list
    self.k = k

  def create_batch(self, true_pairs):
    """
    Create a batch of examples to be entered later to the ip2vec model.
    - pairs_batch: a list of true pairs, containing pairs of entities like described in
      the ip2vec paper [(ip1,ip2)(ip1,destination_protocol2),...].
    - ns_table: A list of different entities from the data, with their empirical
      probabilities, and with the number of times they are to repeat in the table
      that we will train on.

    stages:
    1. Given desired number of examples:
      - Sample alpha in [1, int(theta*<batch_size>)] to make the number of true
        pairs in the batch. The rest of the sample [int(0.25*<batch_size>), (<batch_size>-1)]
        will be the false pairs to be generated.
      - Draw <alpha> number of obs from pairs
      - Draw (<batch_size>-<alpha>) ip addresses from ns_table.
      - Choose with replacement ip addresses from the true pairs and assign them with
        new destinations.

    """
    # unpack from self
    ns_table = self.ns_table
    k = self.k
    batch_size = len(true_pairs)

    # define resulting_batch
    resulting_batch = []

    # add positive pairs to batch
    for pair in true_pairs:
      resulting_batch.append([pair[0],pair[1], 1])

    # keep only sources
    sources = [x[0] for x in true_pairs]

    # Draw batch_size-alpha false entities
    false_entities = np.random.choice(flattened_list, size=(k*batch_size))

    # add negative pairs to batch
    for j in range(len(false_entities)):
      resulting_batch.append([np.random.choice(sources), false_entities[j], 0])

    return np.array(resulting_batch)



In [ ]:
context_generator = ContextGenerator(ns_table=ns_table)
context_generator.create_batch(true_pairs=pairs[:5])


In [ ]:
batch = context_generator.create_batch(true_pairs=pairs[:5])
batch[:,2]

In [ ]:
batch[:,2].astype(int)

In [ ]:
class IP2Vec(nn.Module):
  def __init__(self, vocab_size, word_to_ind, embedding_dim=128):
    super().__init__()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.device = device
    self.word_to_ind = word_to_ind
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.u_embedding = nn.Embedding(vocab_size,embedding_dim)
    self.v_embedding = nn.Embedding(vocab_size,embedding_dim)
    self.log_sigmoid = nn.LogSigmoid()

    init_range= 0.5/embedding_dim
    self.u_embedding.weight.data.uniform_(-init_range,init_range)
    self.v_embedding.weight.data.uniform_(-0,0)

    # Target word embeddings
    self.w = nn.Embedding(voc_size, embedding_dim)
    # Context embeddings
    self.c = nn.Embedding(voc_size, embedding_dim)

  def get_ip_embeddings(self,ip):
    ip = torch.tensor([word_to_ind[ip]])
    ip = ip.to(self.device)
    return self.u_embedding(ip).view(1,-1)

  def forward(self, targets, contexts): # target, context,neg
    # batch looks like the following:
    #   array([[a,b,1],
    #          [c,d,1],
    #          [c,m,0],
    #          [c,g,0]])


    # Look up the embeddings for the target words.
    # shape: (batch size, embedding dimension)
    tagets_embeddings = self.u_embedding(targets)
    n_batch, _ = tagets_embeddings.shape

    # View this as a 3-dimensional tensor, with
    # shape (batch size, 1, embedding dimension)
    tagets_embeddings = tagets_embeddings.view(n_batch, 1, self.embedding_dim)

    # Look up the embeddings for the positive and negative context words.
    # shape: (batch size, nbr contexts, emb dim)
    context_embeddings = self.v_embedding(contexts)

    # Transpose the tensor for matrix multiplication
    # shape: (batch size, emb dim, nbr contexts)
    context_embeddings = context_embeddings.view(n_batch, 1, self.embedding_dim)
    context_embeddings = context_embeddings.transpose(1,2)

    # Compute the dot products between target word embeddings and context
    # embeddings. We express this as a batch matrix multiplication (bmm).
    # shape: (batch size, 1, nbr contexts)
    dots = tagets_embeddings.bmm(context_embeddings)

    # View this result as a 2-dimensional tensor.
    # shape: (batch size, nbr contexts)
    dots = dots.view(n_batch, 1)

    return dots



In [ ]:
print(pairs[:5])
print(np.array([(word_to_ind[x[0]],word_to_ind[x[1]]) for x in pairs[:5]]))



In [ ]:
class SGNSTrainer:
  def __init__(self, pairs, model, ns_table, n_epochs, k, word_to_ind, batch_size=2**10, optimizer='adam'):
    super().__init__()
    """
    ns_table - expanded table


    """
    # Make sure to understand what it means
    # self.instance_gen = instance_gen
    # self.max_words = params.get('max-words')
    # self.n_ns = params['n-neg-samples']

    self.model = model.to(device) # .cuda()#
    self.n_epochs = n_epochs
    self.batch_size = batch_size
    pairs = np.array(pairs)
    self.pairs = pairs
    self.k = k
    self.context_generator = ContextGenerator(ns_table=ns_table, k = self.k)
    self.word_to_ind = word_to_ind

    if optimizer == 'adam':
      self.optimizer = torch.optim.Adam(self.model.parameters())#, lr=params['lr'])
    elif optimizer == 'sgd':
      self.optimizer = torch.optim.SGD(self.model.parameters())#, lr=params['lr'])

    # We'll use a binary cross-entropy loss, since we have a binary classification problem:
    # distinguishing positive from negative contexts.
    self.loss = nn.BCEWithLogitsLoss()
    self.epoch = 0


  def train(self):
    import time
    import matplotlib.pyplot as plt

    def convert_batch_of_words_to_inds(batch, word_to_ind):
        return np.array([(word_to_ind[x[0]],word_to_ind[x[1]], x[2]) for x in batch])

    batch_size = self.batch_size
    t0 = time.time()

    losses_list = []

    for epoch in range(self.n_epochs):
      print(f'Epoch {epoch+1}.')

      pairs = self.pairs.copy()
      np.random.shuffle(pairs)
      j = 0

      # constants for later verbosity
      loss_in_epoch = 0
      n_pairs_epoch = 0

      while (j+1)*batch_size<len(pairs):

        # get a batch of examples
        pairs_in_batch = pairs[(j*batch_size):((j+1)*batch_size)]
        batch_of_examples = context_generator.create_batch(true_pairs=pairs_in_batch)

        # convert from words to inds
        batch_of_examples = convert_batch_of_words_to_inds(batch_of_examples, self.word_to_ind)


        targets = batch_of_examples[:,0].astype(int)
        contexts = batch_of_examples[:,1].astype(int)
        y = batch_of_examples[:,2].astype(int)

        # convert to tensors
        targets = torch.as_tensor(targets).to(device)
        contexts = torch.as_tensor(contexts).to(device)
        y = torch.as_tensor(y).to(device) # .unsqueeze(1)

        # update j
        j+=1

        # clear grads from optimizer
        self.optimizer.zero_grad()

        # Compute the output from the model.
        # That is, the dot products between target embeddings
        # and context embeddings.
        scores = self.model(targets, contexts).squeeze(1)
        loss = self.loss(scores, y.float())

        # Compute gradients and update the embeddings.
        loss.backward()
        self.optimizer.step()

        # We'll print some diagnostics periodically.
        loss_in_epoch += loss.item()
        n_pairs_epoch += (batch_size+1)*self.k
        losses_list.append(loss.item())
        if ((j % 200) == 0):
          print(f'iter {j}, loss {loss.item()}')

      # Print diagnostics
      t1 = time.time()
      print(f'epoch: {epoch}, loss: {loss_in_epoch / n_pairs_epoch:.4f}, time: {t1-t0:.2f}')
      t0 = time.time()

      plt.plot(losses_list)
      plt.title(f'Loss until the end of Epoch {epoch}')
      plt.show()

      self.epoch += 1


In [ ]:
model = None

def main():
    global model

    model = IP2Vec(vocab_size,embedding_dim=128, word_to_ind=word_to_ind)
    trainer = SGNSTrainer(pairs = pairs, model = model, word_to_ind = word_to_ind, batch_size = 2**9, ns_table = ns_table, n_epochs = 2, k = 20)

    trainer.train()

main()


In [ ]:
model.get_ip_embeddings('24.251.45.232')


In [ ]:
model.get_ip_embeddings('109.66.135.125')
# model.get_ip_embeddings('36982')


In [ ]:
from sklearn.manifold import TSNE
from sklearn import preprocessing

def get_embeddings(ip):
  s = model.get_ip_embeddings(ip)
  return preprocessing.normalize(s.cpu().detach().numpy(), norm='l2')

entities = [x[0] for x in ns_table]

vectors_of_embeddings = np.array([get_embeddings(x) for x in entities]).reshape(len(entities), 128)
print(vectors_of_embeddings.shape)

print('Example of normalized embeddings:')
print(get_embeddings(entities[7]))

for perplexity_ in [25,50,75,100,200]:
  tsne_x = TSNE(n_components=2, perplexity=perplexity_).fit_transform(vectors_of_embeddings)
  plt.figure(figsize=(10,10))
  plt.scatter(tsne_x[:,0], tsne_x[:,1])
  plt.title(f't-SNE plot with perplexity of {perplexity_}')
  plt.show()